# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='Reacher.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.0


When finished, you can close the environment.

In [6]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [5]:
from Agent import DDPGagent
from collections import deque
import matplotlib.pyplot as plt
import seaborn as sns
from Utils import *
import numpy as np
import time

sns.set()
%matplotlib inline

In [6]:
agent = DDPGagent(state_size=state_size, action_size=action_size, random_seed=2)

def DDPG(n_episodes=2000, max_t=10000, print_every=1):
    """
    DDPG 
    
    Parameters
    ==========
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        
    """
    # list containing scores from each episode
    scores = []      
    # last 100 scores
    scores_window = deque(maxlen=100)           
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        agent.reset()
        start_time = time.time()
        
        for t in range(max_t):
            # select an action
            action = agent.act(state, add_noise=True)
            # send the action to the environment
            env_info = env.step(action)[brain_name]        
            # get the next state
            next_state = env_info.vector_observations[0]
            # obtain the reward
            reward = env_info.rewards[0]
            # see whether or not the episode is finished
            done = env_info.local_done[0]
            agent.step(state, action, reward, next_state, done)
            score += reward
            # roll over to the next timestep
            state = next_state
            if done:
                break 
                
        duration = time.time() - start_time
        # save most recent score        
        scores_window.append(score) 
        # save most recent score
        scores.append(score)             
        
        # print out some statistics
        if i_episode % print_every == 0:
            print('\rEpisode {} ({} seconds)\tScore: {:.2f}\tAverage Score: {:.2f}'.format(
                i_episode, duration, score, np.mean(scores_window)))
            torch.save(agent.actor.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic.state_dict(), 'checkpoint_critic.pth')
        
        if np.mean(scores_window)>=30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            torch.save(agent.actor.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic.state_dict(), 'checkpoint_critic.pth')
            break
            
    return scores

scores = DDPG()

Episode 1 (17.73602604866028 seconds)	Score: 0.10	Average Score: 0.10
Episode 2 (19.4415500164032 seconds)	Score: 2.39	Average Score: 1.24
Episode 3 (17.15985608100891 seconds)	Score: 0.75	Average Score: 1.08
Episode 4 (16.699329137802124 seconds)	Score: 0.67	Average Score: 0.98
Episode 5 (20.30038809776306 seconds)	Score: 1.62	Average Score: 1.11
Episode 6 (17.20131206512451 seconds)	Score: 0.51	Average Score: 1.01
Episode 7 (16.911168813705444 seconds)	Score: 0.21	Average Score: 0.89
Episode 8 (16.56749963760376 seconds)	Score: 0.17	Average Score: 0.80
Episode 9 (16.465851068496704 seconds)	Score: 0.10	Average Score: 0.72
Episode 10 (16.420358180999756 seconds)	Score: 0.29	Average Score: 0.68
Episode 11 (16.342376947402954 seconds)	Score: 1.26	Average Score: 0.73
Episode 12 (16.453145027160645 seconds)	Score: 0.57	Average Score: 0.72
Episode 13 (16.34140181541443 seconds)	Score: 0.00	Average Score: 0.66
Episode 14 (16.308971166610718 seconds)	Score: 0.00	Average Score: 0.62
Episode 1

Episode 116 (17.240015745162964 seconds)	Score: 2.05	Average Score: 2.26
Episode 117 (17.02346706390381 seconds)	Score: 2.60	Average Score: 2.28
Episode 118 (17.111409187316895 seconds)	Score: 4.24	Average Score: 2.32
Episode 119 (17.111214876174927 seconds)	Score: 3.60	Average Score: 2.34
Episode 120 (17.299484252929688 seconds)	Score: 3.83	Average Score: 2.37
Episode 121 (17.202933073043823 seconds)	Score: 2.62	Average Score: 2.40
Episode 122 (17.103217124938965 seconds)	Score: 3.44	Average Score: 2.41
Episode 123 (17.203696250915527 seconds)	Score: 1.87	Average Score: 2.40
Episode 124 (17.09867811203003 seconds)	Score: 4.38	Average Score: 2.43
Episode 125 (17.392296075820923 seconds)	Score: 2.83	Average Score: 2.45
Episode 126 (17.591500997543335 seconds)	Score: 3.58	Average Score: 2.48
Episode 127 (17.135204792022705 seconds)	Score: 2.71	Average Score: 2.48
Episode 128 (17.308748722076416 seconds)	Score: 3.92	Average Score: 2.51
Episode 129 (17.16094994544983 seconds)	Score: 7.18	A

Episode 229 (17.62847590446472 seconds)	Score: 3.28	Average Score: 4.09
Episode 230 (17.07138991355896 seconds)	Score: 3.10	Average Score: 4.10
Episode 231 (17.1200749874115 seconds)	Score: 2.34	Average Score: 4.07
Episode 232 (17.11678123474121 seconds)	Score: 5.68	Average Score: 4.11
Episode 233 (17.015679836273193 seconds)	Score: 0.29	Average Score: 4.05
Episode 234 (17.184056282043457 seconds)	Score: 3.60	Average Score: 4.06
Episode 235 (17.105988025665283 seconds)	Score: 5.77	Average Score: 4.08
Episode 236 (17.143922090530396 seconds)	Score: 5.02	Average Score: 4.09
Episode 237 (17.03147602081299 seconds)	Score: 5.43	Average Score: 4.09
Episode 238 (17.10442280769348 seconds)	Score: 6.06	Average Score: 4.10
Episode 239 (17.212265968322754 seconds)	Score: 6.53	Average Score: 4.13
Episode 240 (16.90034770965576 seconds)	Score: 3.88	Average Score: 4.12
Episode 241 (17.04072403907776 seconds)	Score: 4.46	Average Score: 4.10
Episode 242 (17.07845973968506 seconds)	Score: 5.71	Average 

Episode 342 (17.22791814804077 seconds)	Score: 3.33	Average Score: 4.59
Episode 343 (17.225487232208252 seconds)	Score: 5.61	Average Score: 4.59
Episode 344 (17.272562265396118 seconds)	Score: 3.05	Average Score: 4.55
Episode 345 (17.133655071258545 seconds)	Score: 7.35	Average Score: 4.56
Episode 346 (17.06806182861328 seconds)	Score: 3.43	Average Score: 4.54
Episode 347 (17.20345401763916 seconds)	Score: 2.27	Average Score: 4.50
Episode 348 (17.119927883148193 seconds)	Score: 2.77	Average Score: 4.49
Episode 349 (17.230488061904907 seconds)	Score: 3.11	Average Score: 4.47
Episode 350 (17.214109182357788 seconds)	Score: 4.97	Average Score: 4.45
Episode 351 (17.603211164474487 seconds)	Score: 4.13	Average Score: 4.43
Episode 352 (17.07777690887451 seconds)	Score: 2.21	Average Score: 4.43
Episode 353 (17.074446201324463 seconds)	Score: 6.14	Average Score: 4.47
Episode 354 (17.12962317466736 seconds)	Score: 3.16	Average Score: 4.44
Episode 355 (17.24017572402954 seconds)	Score: 3.72	Aver

Episode 455 (17.630979776382446 seconds)	Score: 5.29	Average Score: 4.72
Episode 456 (17.29788088798523 seconds)	Score: 5.67	Average Score: 4.72
Episode 457 (17.173027992248535 seconds)	Score: 4.37	Average Score: 4.71
Episode 458 (17.100666999816895 seconds)	Score: 5.86	Average Score: 4.73
Episode 459 (17.093801021575928 seconds)	Score: 5.37	Average Score: 4.75
Episode 460 (17.195487022399902 seconds)	Score: 4.89	Average Score: 4.74
Episode 461 (17.202678203582764 seconds)	Score: 13.84	Average Score: 4.83
Episode 462 (17.064868927001953 seconds)	Score: 6.52	Average Score: 4.84
Episode 463 (17.05729365348816 seconds)	Score: 3.86	Average Score: 4.84
Episode 464 (17.009440898895264 seconds)	Score: 6.56	Average Score: 4.82
Episode 465 (17.44762682914734 seconds)	Score: 7.10	Average Score: 4.84
Episode 466 (17.252306938171387 seconds)	Score: 4.54	Average Score: 4.84
Episode 467 (17.195836782455444 seconds)	Score: 5.39	Average Score: 4.83
Episode 468 (17.300331115722656 seconds)	Score: 4.44	

...

Episode 1468 (17.46943211555481 seconds)	Score: 4.89	Average Score: 3.54
Episode 1469 (17.510043144226074 seconds)	Score: 1.79	Average Score: 3.52
Episode 1470 (17.459877967834473 seconds)	Score: 4.46	Average Score: 3.55
Episode 1471 (17.535197019577026 seconds)	Score: 2.45	Average Score: 3.53
Episode 1472 (17.613599061965942 seconds)	Score: 1.81	Average Score: 3.51
Episode 1473 (17.727349996566772 seconds)	Score: 4.04	Average Score: 3.52
Episode 1474 (17.628482818603516 seconds)	Score: 5.86	Average Score: 3.55
Episode 1475 (17.59200692176819 seconds)	Score: 4.80	Average Score: 3.55
Episode 1476 (17.64043402671814 seconds)	Score: 3.19	Average Score: 3.58
Episode 1477 (17.57435178756714 seconds)	Score: 2.56	Average Score: 3.56
Episode 1478 (17.588923931121826 seconds)	Score: 3.90	Average Score: 3.58
Episode 1479 (17.50356388092041 seconds)	Score: 4.24	Average Score: 3.57
Episode 1480 (17.634624004364014 seconds)	Score: 4.84	Average Score: 3.59
Episode 1481 (17.429720878601074 seconds)	S

Episode 1580 (17.701515913009644 seconds)	Score: 2.08	Average Score: 2.98
Episode 1581 (17.599699020385742 seconds)	Score: 1.94	Average Score: 2.98
Episode 1582 (17.449726343154907 seconds)	Score: 2.08	Average Score: 2.96
Episode 1583 (17.41440987586975 seconds)	Score: 2.76	Average Score: 2.94
Episode 1584 (18.29454493522644 seconds)	Score: 3.99	Average Score: 2.94
Episode 1585 (17.99489688873291 seconds)	Score: 2.89	Average Score: 2.95
Episode 1586 (17.778738975524902 seconds)	Score: 5.82	Average Score: 2.96
Episode 1587 (17.35636591911316 seconds)	Score: 2.39	Average Score: 2.94
Episode 1588 (17.47036385536194 seconds)	Score: 2.15	Average Score: 2.92
Episode 1589 (17.64411497116089 seconds)	Score: 1.55	Average Score: 2.94
Episode 1590 (17.627013206481934 seconds)	Score: 4.03	Average Score: 2.93
Episode 1591 (17.65869903564453 seconds)	Score: 2.44	Average Score: 2.93
Episode 1592 (17.641613006591797 seconds)	Score: 2.00	Average Score: 2.92
Episode 1593 (17.668988943099976 seconds)	Sco

Episode 1692 (17.704022884368896 seconds)	Score: 2.83	Average Score: 2.22
Episode 1693 (17.589534044265747 seconds)	Score: 2.99	Average Score: 2.23
Episode 1694 (17.51183295249939 seconds)	Score: 3.06	Average Score: 2.24
Episode 1695 (17.58913493156433 seconds)	Score: 2.67	Average Score: 2.24
Episode 1696 (18.140026092529297 seconds)	Score: 1.58	Average Score: 2.25
Episode 1697 (18.573082208633423 seconds)	Score: 4.52	Average Score: 2.27
Episode 1698 (17.647599935531616 seconds)	Score: 5.80	Average Score: 2.30
Episode 1699 (18.18141794204712 seconds)	Score: 0.52	Average Score: 2.29
Episode 1700 (17.684854984283447 seconds)	Score: 1.42	Average Score: 2.30
Episode 1701 (17.61982297897339 seconds)	Score: 1.71	Average Score: 2.28
Episode 1702 (17.679124116897583 seconds)	Score: 1.73	Average Score: 2.29
Episode 1703 (17.931347131729126 seconds)	Score: 3.25	Average Score: 2.29
Episode 1704 (17.696364164352417 seconds)	Score: 1.40	Average Score: 2.27
Episode 1705 (17.612807989120483 seconds)	

Episode 1804 (17.599659204483032 seconds)	Score: 1.70	Average Score: 2.58
Episode 1805 (17.884124040603638 seconds)	Score: 1.50	Average Score: 2.59
Episode 1806 (17.496186017990112 seconds)	Score: 1.75	Average Score: 2.59
Episode 1807 (17.62896203994751 seconds)	Score: 3.27	Average Score: 2.61
Episode 1808 (17.60066294670105 seconds)	Score: 3.58	Average Score: 2.61
Episode 1809 (17.618574142456055 seconds)	Score: 4.67	Average Score: 2.65
Episode 1810 (17.48148512840271 seconds)	Score: 0.81	Average Score: 2.64
Episode 1811 (17.545533180236816 seconds)	Score: 1.91	Average Score: 2.63
Episode 1812 (17.743231058120728 seconds)	Score: 1.19	Average Score: 2.61
Episode 1813 (17.584864854812622 seconds)	Score: 2.62	Average Score: 2.60
Episode 1814 (17.630500078201294 seconds)	Score: 1.76	Average Score: 2.60
Episode 1815 (17.54402208328247 seconds)	Score: 2.67	Average Score: 2.59
Episode 1816 (17.641925811767578 seconds)	Score: 2.48	Average Score: 2.58
Episode 1817 (17.546004056930542 seconds)	

Episode 1916 (17.618299961090088 seconds)	Score: 1.99	Average Score: 2.77
Episode 1917 (17.531677961349487 seconds)	Score: 1.05	Average Score: 2.76
Episode 1918 (17.633715867996216 seconds)	Score: 4.12	Average Score: 2.78
Episode 1919 (17.670386791229248 seconds)	Score: 5.51	Average Score: 2.79
Episode 1920 (17.54940176010132 seconds)	Score: 4.15	Average Score: 2.81
Episode 1921 (18.47899317741394 seconds)	Score: 2.30	Average Score: 2.80
Episode 1922 (18.436729669570923 seconds)	Score: 2.80	Average Score: 2.80
Episode 1923 (18.605303049087524 seconds)	Score: 2.07	Average Score: 2.80
Episode 1924 (17.57517099380493 seconds)	Score: 0.80	Average Score: 2.79
Episode 1925 (17.664075136184692 seconds)	Score: 3.01	Average Score: 2.80
Episode 1926 (17.737173080444336 seconds)	Score: 1.68	Average Score: 2.79
Episode 1927 (17.574612140655518 seconds)	Score: 1.21	Average Score: 2.77
Episode 1928 (17.654705047607422 seconds)	Score: 2.14	Average Score: 2.77
Episode 1929 (17.694175958633423 seconds)